In [1]:
import yfinance as yf
import yahoo_fin.stock_info as si
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
ticker = "NVDA"
nvda = yf.Ticker(ticker)
# mean( get.current.return('^GSPC', startDate, endDate), na.rm=T ) * 252
# [1] 0.1032009
sp500Annualret = 0.1032009
riskFreeRate2024 = 0.0458

In [29]:
costOfEquity = riskFreeRate2024 + nvda.info['beta'] * (sp500Annualret - riskFreeRate2024)
totalDebt = nvda.info['totalDebt']
marketCap = nvda.info['marketCap']
totalValue = marketCap + totalDebt
nvdaFinancialStatement = nvda.financials
interest = nvdaFinancialStatement.loc["Interest Expense"][0]
costOfEquity = riskFreeRate2024 + nvda.info['beta'] * (sp500Annualret - riskFreeRate2024)
interestRate = interest / totalDebt
taxRate2024 = nvdaFinancialStatement.loc["Tax Provision"][0] / nvdaFinancialStatement.loc["Pretax Income"][0]
costOfDebt = interestRate 
WACC = ( ( marketCap / totalValue ) * costOfEquity ) + ( (totalDebt / totalValue ) * costOfDebt * ( 1 - taxRate2024 ) )
print(f"{ticker} 2024 wacc: {WACC:.2%}")

NVDA 2024 wacc: 14.18%


In [79]:
Tick = yf.Ticker("GLD")
Tick.cash_flow


""


In [7]:
def getWACC(ticker, riskFreeRate2024 = 0.0458, sp500Annualret = 0.1032009):
    """ Calculate the Weighted Average Cost of Capital (WACC) for a company.
    Args:
        ticker (str): string of company ticker
        riskFreeRate2024 (float): risk free rate in 2024
        sp500Annualret (float): Annual returns from sp500. Defaults to 0.1032009.
    Returns:
        float: WACC
    """

    Tick = yf.Ticker(ticker)

    try:
        costOfEquity = riskFreeRate2024 + Tick.info['beta'] * (sp500Annualret - riskFreeRate2024)
        FinancialStatement = Tick.financials
        totalDebt = Tick.info['totalDebt']
        marketCap = Tick.info['marketCap']

        totalValue = marketCap + totalDebt
        interest = FinancialStatement.loc["Interest Expense"][0]
        costOfEquity = riskFreeRate2024 + Tick.info['beta'] * (sp500Annualret - riskFreeRate2024)
        interestRate = interest / totalDebt
        taxRate2024 = FinancialStatement.loc["Tax Provision"][0] / FinancialStatement.loc["Pretax Income"][0]

        costOfDebt = interestRate 
        WACC = ( ( marketCap / totalValue ) * costOfEquity ) + ( (totalDebt / totalValue ) * costOfDebt * ( 1 - taxRate2024 ) )
        return WACC
    except: 
        return None

In [4]:
getWACC('AAPL')

0.11474299012837469

In [9]:
tickers = ["NVDA", "AMD", "TSM", "ASML", "MU", "AMAT",
        "GOOG", "AAPL", "META", "TSLA", "DUK",
        "GLD", "X", "PG", "TLT","CVS", "VGIT", "GOVT"]

wacc_list = []

for ticker in tickers:
    print(ticker)
    wacc = getWACC(ticker)
    wacc_list.append(wacc)

pd.DataFrame({"Ticker": tickers,
              "wacc": wacc_list}).to_csv("wacc_list.csv")

NVDA
AMD
TSM
ASML
MU
AMAT
GOOG
AAPL
META
TSLA
DUK
GLD
X
PG
TLT
CVS
VGIT
GOVT
